By Amy Weng

This code experiments with sentiment analysis on a small dataset. 

References: 
https://github.com/ABeeShake/Ethical-Consumption-Before-Capitalism/blob/main/Sentiment%20Analysis/SATM_full_aws.py

https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/

In [1]:
import pandas as pd
import os 
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
def wordsearch(keyword, n, text):
    words = text.split()
    fullwords = ""
    
    for index, word in enumerate(words):
        if word == keyword:
            left_side = words[index - n: index]
            right_side = words[index: index + n + 1]  
            left = ' '.join(left_side)
            right = ' '.join(right_side)
            fullwords = fullwords + left + " " + right + " "
    return fullwords

In [11]:
# new code for finding text windows + particular text info for manual review 
def find_text_window(check,keywords, f, df):
    words = keywords.split("|")
    count=0
    text = df.iloc[0].text

    if (re.search(check, text) != None):
        if count==0:
            a = df.iloc[0].author
            p = df.iloc[0].publisher
            d = df.iloc[0].date
            t = df.iloc[0].title 
            f.write(str(d)+'\n'+str(t)+'\n'+'author: '+str(a)+'\n'+'publisher: '+str(p)+'\n')
        else: 
            count=1
        for k in range(len(words)):
            surrounding = wordsearch(str(words[k]), 15, text)
            if (surrounding != None):
                f.write(surrounding)
    f.write('\n\n')

def excerpt(word, infile,outfile):
    check = re.compile(word)
    readFile = pd.read_csv(infile) 

    f = open(outfile, "a+")

    for i in range(len(readFile.index)):
        df = readFile[i:(i+1)]
        find_text_window(check,word,f,df)

    f.close()

In [ ]:
# Sentiment Analysis Code 
 
def make_text_window(check,keywords, inFile, i, text):
    words = keywords.split("|")

    if (re.search(check, text) != None):
        newFile = inFile + '_' + str(i) + ('.txt')
        fullName =  '/home/rapiduser/Materials/sentiment analysis/'+newFile
        f = open(fullName, "a+")

        for k in range(len(words)):
            surrounding = wordsearch(str(words[k]), 15, text)
            if (surrounding != None):
                f.writelines(surrounding)
        
        f.close()

SIA = SentimentIntensityAnalyzer()

def sentiment_scores(scores, text,file):
	
	sentiment_dict = SIA.polarity_scores(text)

	if sentiment_dict['compound'] >= 0.05 :
		scores["Positive"] += 1
		# print('positive ' + file)

	elif sentiment_dict['compound'] <= - 0.05 :
		scores["Negative"] += 1
		# print('negative ' + file)

	else :
		scores["Neutral"] += 1
		# print('positive ' + file)

In [ ]:
def sentiment(folder,file,term):
    check = re.compile(term)
    readFile = pd.read_csv(file) 

    for i in range(len(readFile.index)):
        df = readFile[i:(i+1)]
        print(df.iloc[0].title)
        text = df.iloc[0].text
        make_text_window(check,term,"file",i,text)

    scores = {"Positive":0,"Negative":0,"Neutral":0}
    for file in os.listdir(folder):
        if (file.endswith(".txt")):
            with open(folder+"/"+file,'r') as f:
                data1 = f.read()
                sentiment_scores(scores,data1,file)
    print(scores)